Tensorflow 에서는 정해진 사이즈의 큐를 중심으로 큐에 데이터를 넣는 행위와 빼서 사용하는 행위를 멀티 쓰레드 기반으로 수행할 수 있도록 라이브러리를 제공하고 있다. 실제로 대량의 데이터를 Tensowflow 를 통해서 훈련하고자 할 경우 발생할 수 있는 메모리 오버플러우 등을 회피 할 수 있는 효과적인 수단으로 사용될 수 있다. 어떤 식으로 Queue 를 사용하여 Thread 처리를 하는 지 예제를 통해서 살펴 보도록 하자 

In [2]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(['./data/test.csv'])
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
record_defaults = [[1], [1], [1], [1], [1]]
col1, col2, col3, col4, col5 = tf.decode_csv(value, record_defaults=record_defaults)
feature = tf.stack([col1, col2, col3, col4, col4, col5])

with tf.Session() as sess :
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
      
    for i in range(10):
        example, label = sess.run([feature, col5])     
        print("data : {0} , lable : {1}".format(example,label))
                   
    coord.request_stop()
    coord.join(threads)

data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5
data : [1 2 3 4 4 5] , lable : 5


In [ ]:
import tensorflow as tf


# filename_pattern = '/home/dev/hoyai_jupyter/chap02_data_handling/data/*.csv'
# filename_list = tf.train.match_filenames_once(filename_pattern)
# filename_queue = tf.train.string_input_producer(filename_list, shuffle=True, num_epochs=2)
# filename_queue = tf.train.string_input_producer(['data/test.csv','data/test.csv'])

filename_queue = tf.train.string_input_producer(['./data/test.csv'])

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)

record_defaults = [[1], [1], [1], [1], [1]]
col1,col2,col3,col4,col5 = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.stack([col1,col2,col3,col4,col5])

# min_after_dequeue = 5
# capacity = min_after_dequeue + 3 * batch_size
# queue = tf.RandomShuffleQueue(
#     capacity=capacity, 
#     min_after_dequeue=min_after_dequeue, 
#     dtypes=tf.int32, 
#     shapes=[5,]
# )
#enqueue_op = queue.enqueue(features)

batch_size = 1
queue = tf.FIFOQueue(10, tf.int32, shapes=[5,])
enqueue_op = queue.enqueue(features)
inputs = queue.dequeue_many(batch_size)
#tf.tables_initializer()
qr = tf.train.QueueRunner(queue, [enqueue_op] * 2)


sess = tf.Session()
sess.run(tf.initialize_all_variables())
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

try:
    for step in range(10):
        print(step)
        if coord.should_stop():
            print("stop")
            break
        print(sess.run(features))

except Exception as e:
    print(e)
    coord.request_stop(e)

coord.request_stop()
coord.join(enqueue_threads)



0


In [ ]:
import numpy as np
import tensorflow as tf
import threading

BATCH_SIZE = 5
TRAINING_ITERS = 100

# Features are length-100 vectors of floats
feature_input = tf.placeholder(tf.float32, shape=[100])
# Labels are scalar integers.
label_input = tf.placeholder(tf.int32, shape=[])

# Alternatively, could do:
# feature_batch_input = tf.placeholder(tf.float32, shape=[None, 100])
# label_batch_input = tf.placeholder(tf.int32, shape=[None])

q = tf.FIFOQueue(100, [tf.float32, tf.int32], shapes=[[100], []])
enqueue_op = q.enqueue([label_input, feature_input])

# For batch input, do:
# enqueue_op = q.enqueue_many([label_batch_input, feature_batch_input])

label_batch, feature_batch = q.dequeue_many(BATCH_SIZE)

sess = tf.Session()

def load_and_enqueue():
  with open(...) as feature_file, open(...) as label_file:
    while True:
      feature_array = numpy.fromfile(feature_file, numpy.float32, 100)
      if not feature_array:
        return
      label_value = numpy.fromfile(feature_file, numpy.int32, 1)[0]

      sess.run(enqueue_op, feed_dict={feature_input: feature_array,
                                      label_input: label_value})

# Start a thread to enqueue data asynchronously, and hide I/O latency.
t = threading.Thread(target=load_and_enqueue)
t.start()

In [1]:
# Initialize placeholders for feeding in to the queue
pl_queue_screens = tf.placeholder(tf.float32, shape=[config.seq_length, config.image_size, config.image_size, config.input_channels], name="queue_inputs")
pl_queue_targets = tf.placeholder(tf.uint8, shape=[config.seq_length], name="queue_targets_cnt")

#  ...

capacity = config.min_after_dequeue + 10 * (config.num_gpus*config.batch_size)
q = tf.RandomShuffleQueue(
    capacity=capacity,
    min_after_dequeue=config.min_after_dequeue,
    dtypes=[tf.float32, tf.uint8],
    shapes=[[config.seq_length, config.image_size, config.image_size, config.input_channels], [config.seq_length]]
)


# ..and finally the enqueue operation for adding a single sequence
enqueue_op = q.enqueue([seq_proc, pl_queue_targets])

# Misc queue operations
examples_in_queue = q.size()
queue_close_op = q.close(cancel_pending_enqueues=True)

# This must be the input for the training operation
inputs_batch_queue, targets_batch_queue = q.dequeue_many(config.batch_size)

# Placeholders for training and evaluation
batch_screens = tf.placeholder_with_default(inputs_batch_queue, [None, config.seq_length, config.image_size, config.image_size, config.input_channels], name="model_inputs")
batch_targets = tf.placeholder_with_default(targets_batch_queue, [None, config.seq_length], name="model_targets_cnt")
dropout_keep_prob = tf.placeholder_with_default(tf.constant(1.0), shape=[], name="dropout_keep_prob")

# ...

########################################################################################################
########################################################################################################
########################################################################################################

# Now we start a number of threads that read from disk (numpy) array and feed it to the queue
# Coordinator for threads
coord = tf.train.Coordinator()

# Start the data loading + preprocessing threads
threads = []
for _ in range(config.num_preproc_threads):
	
	# This is the method that runs in the threads and feeds examples to the queue
    t = threading.Thread(target=load_preproc_enqueue_thread, args=(
        sess, coord, enqueue_op, pl_queue_screens, pl_queue_targets,
    	# additional arguments ...     
    ))

    t.setDaemon(True)
    t.start()
    threads.append(t)
    coord.register_thread(t)
    time.sleep(0.5)

num_examples_in_queue = sess.run(examples_in_queue)
while num_examples_in_queue < config.min_after_dequeue:
    num_examples_in_queue = sess.run(examples_in_queue)
    for t in threads:
        if not t.isAlive():
            coord.request_stop()
            raise ValueError("One or more enqueuing threads crashed...")

    print("Filling up queue with training examples: %i/%i" % (num_examples_in_queue, config.min_after_dequeue))
    time.sleep(1)

# ...

########################################################################################################
########################################################################################################
########################################################################################################

# ...
# For your training operation use batch_screens and batch_targets as inputs
# Look at the use of tf.placeholder_with_default() => if no feed_dict{} is fed in then the placeholder will fetch examples from the queue
# For your validation data, you can just use the placeholders/feeddict.


########################################################################################################


def load_preproc_enqueue_thread(sess, coord, enqueue_op, pl_queue_screens, pl_queue_targets):
	
	# MODIFY THIS FUNCTION FOR LOADING SLICES FROM YOUR INPUT TENSOR AND FEEDING INTO QUEUE PLACEHOLDERS

    # Fetch the HDF5 files from disk...
    filenames_queue = glob.glob(os.path.join(dataset_path, "train/*.h5"))
    filenames_queue.sort()

    while not coord.should_stop():

        # Shuffle the filenames each time we have fed everything
        random.shuffle(filenames_queue)

        for filename in filenames_queue:

            # Read 100 examples from HDF5 file, shuffle files within file.
            # Optionally also perform mean subtraction and normalization.
            # Sequences can also be tiled if the examples are padded with zeros
            # after one cycle length.
            screens, _, count_labels, cycle_lengths, residual_frames, _ = \
                read_examples_from_hdf5(
                    filename, shuffle=True, data_whitening=data_whitening,
                    expand_dims=expand_dims, convert_to_grayscale=convert_to_grayscale
                )

            # Feed Dictionary without the labels
            feed_dict = {
                pl_queue_screens: screens[index,],
                pl_queue_targets: targets,
                pl_flip_image: random_flip,
                pl_brightness_delta: random_brightness_delta,
                pl_contrast_factor: random_contrast_factor
            }

            # Feed examples to the queue
            try:
                sess.run(enqueue_op, feed_dict=feed_dict)
            except tf.errors.CancelledError:
                return

NameError: name 'tf' is not defined